# Parameters

In [1]:
year = 2022
week = 22

# Nombre del archivo a importar de Sell Out
file_import_so = 'Walmartnuevo_05062022.zip'
# Nombre del archivo a importar de Stock
file_import_stock = 'WalmartStapleStock_01042022.xls'
# Nombre del archivo a exportar de Sell Out
file_export_so = 'Walmartnuevo.txt'
# Nombre del archivo a exportar de Stock
file_export_stock = 'WalmartStapleStock.txt'

In [2]:
# Directorio de donde se importarán los archivos
dir_files_wm = r'C:\Users\jshernandezm\OneDrive - genommalabinternacional\MEX\Carga\Diarios\WM\Data\{0}\{1}\Raw'

# Directorio donde se exportará el output
dir_backup = r'C:\Users\jshernandezm\OneDrive - genommalabinternacional\MEX\Carga\Diarios\WM\Data\{0}\{1}\Output'

# Libraries

In [3]:
from zipfile import ZipFile
import pandas as pd
import os


# Zip File

Descomprimimos el archivo Sell Out y guardamos el txt en la misma ruta:

In [4]:
# Agregamos el año y ruta en el directorio
dir_files_wm = dir_files_wm.format(year, str(week).zfill(2))
# Extracción de archivo
with ZipFile(dir_files_wm + '\\' + file_import_so, 'r') as zip_ref:
    zip_ref.extractall(dir_files_wm)
# Removemos el objeto zip_ref
del zip_ref

Obtenemos el nombre del archivo que acabamos de descomprimir:

In [5]:
file_import_so = [file for file in os.listdir(dir_files_wm) if file.endswith('.txt')][0]

# Import Data

Cargamos el archivo de Sell Out para quitarle los caracteres raros ()

In [6]:
df_so = pd.read_csv(
    dir_files_wm + '\\' + file_import_so, 
    sep='\t', 
    encoding='windows-1252',
    header=None,
    dtype={0:str}
)

# Data Cleaning

Removemos los caracteres de interés:

In [7]:
df_so[4] = df_so[4].apply(lambda x: x.replace('AÂ¨AÂ¨', ''))
df_so[4] = df_so[4].apply(lambda x: x.replace('Â', ''))
df_so[4] = df_so[4].apply(lambda x: x.replace('ï¿½', ''))

Y también quitamos aquellos registros que contengan *216pc* o *234PCGOICOECHEA ANT*:

In [21]:
mask = df_so[4].apply(lambda x: ('216pc' in x) or ('234PCGOICOECHEA ANT' in x))
df_so = df_so[~mask]

Transformamos las últimas 7 columnas a string con 6 decimales:

In [22]:
cols = range(11, 18)
for col in cols:
    df_so[col] = df_so[col].apply(lambda x: '%.6f'%(x))

C:\Users\jshernandezm\AppData\Local\Temp\ipykernel_19320\2794966775.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_so[col] = df_so[col].apply(lambda x: '%.6f'%(x))


Generamos una función para quedarnos con la parte entera y respete las filas nulas

In [23]:
def get_integers(num):
    try:
        return '%.0f'%(num)
    except:
        return num

In [24]:
df_so[9] = df_so[9].map(get_integers)

C:\Users\jshernandezm\AppData\Local\Temp\ipykernel_19320\262723946.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_so[9] = df_so[9].map(get_integers)


# Export txt

Se exporta el archivo txt a la ruta backup:

In [25]:
dir_backup = dir_backup.format(year, str(week).zfill(2))
df_so.to_csv(
    dir_backup + '\\' + file_export_so,
    sep='\t',
    encoding='latin1',
    header=None,
    index=False
)